### All the Required Imports Goes in to this Cell

In [7]:
import polars as pl
from polars import DataFrame
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt

### Reading the input dataset `prices-split-adjusted.csv`

In [11]:
df: DataFrame = pl.read_csv("prices-split-adjusted.csv")
df.head()

date,symbol,open,close,low,high,volume
str,str,f64,f64,f64,f64,f64
"""2016-01-05""","""WLTW""",123.43,125.839996,122.309998,126.25,2.1636e6
"""2016-01-06""","""WLTW""",125.239998,119.980003,119.940002,125.540001,2.3864e6
"""2016-01-07""","""WLTW""",116.379997,114.949997,114.93,119.739998,2.4895e6
"""2016-01-08""","""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
"""2016-01-11""","""WLTW""",117.010002,114.970001,114.089996,117.330002,1.4086e6


### Converting the date from `string` to actual `date` type

In [12]:
df = df.with_columns([
    pl.col("date").str.to_date().alias("date")
])
df.head()

date,symbol,open,close,low,high,volume
date,str,f64,f64,f64,f64,f64
2016-01-05,"""WLTW""",123.43,125.839996,122.309998,126.25,2.1636e6
2016-01-06,"""WLTW""",125.239998,119.980003,119.940002,125.540001,2.3864e6
2016-01-07,"""WLTW""",116.379997,114.949997,114.93,119.739998,2.4895e6
2016-01-08,"""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
2016-01-11,"""WLTW""",117.010002,114.970001,114.089996,117.330002,1.4086e6


### Filling Missing date records with forward fill

In [15]:
df = df.set_sorted("date").upsample(time_column="date",every="1d").select(pl.all().forward_fill())
df.head(10)

date,symbol,open,close,low,high,volume
date,str,f64,f64,f64,f64,f64
2016-01-05,"""WLTW""",123.43,125.839996,122.309998,126.25,2.1636e6
2016-01-06,"""WLTW""",125.239998,119.980003,119.940002,125.540001,2.3864e6
2016-01-07,"""WLTW""",116.379997,114.949997,114.93,119.739998,2.4895e6
2016-01-08,"""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
2016-01-09,"""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
2016-01-10,"""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
2016-01-11,"""WLTW""",117.010002,114.970001,114.089996,117.330002,1.4086e6
2016-01-12,"""WLTW""",115.510002,115.550003,114.5,116.059998,1.098e6
2016-01-13,"""WLTW""",116.459999,112.849998,112.589996,117.07,949600.0
